# PRODUCTION ALLOCATION PROBLEM
__DHARINI SUDARSAN \[245997\]__

## Introduction
Problem 16 in [opt_LPlab2_2021](http://prac.im.pwr.wroc.pl/~wiecek/opt_LPlab2_2021.pdf).

A production house has 10 cells and produces 5 products. Each cell can produce only certain products. Given the product requirement and production rate of each product, we aim to determine minimum number of cells to be opened.  
This problem is solved using puLP library in python and numpy library is used for defining variables.

In [1]:
from pulp import *
import numpy as np

## Problem Definition
### Parameters    
The following constant parameters are defined with the given information in the problem.  
$W_{p\ c}$ denotes whether cell $c$ is able to  produce(1) or not produce(0) product $p$  
$D_{p}$ denotes demand requirement for product $p$  
$T_{p}$ denotes production rate per hour product $p$  
$n_{p}$ denotes number of products  
$n_{c}$ denotes number of cells  

In [2]:
W = np.array([
              [1,1,0,0,1,1,0,1,0,1],
              [1,0,1,0,1,0,1,0,1,1],
              [1,1,0,1,0,0,1,1,1,0],
              [0,1,0,0,1,1,1,0,1,0],
              [0,1,0,0,1,1,0,1,1,0]
              ])
D = np.array([11,17,16,15,12])
T = np.array([9,11,10,8,6])
n_cells = 10
n_products = 5

### Decision Variables
#### Cell operation variables
The cell operation variable is an indicator variable that denotes whether cell is open/close.
$Z_{c} \ =\ \begin{cases}
1 & \mathrm{if,} \ cell\ c\ is\ open\\
0 & \mathrm{if,} \ cell\ c\ is\ closed\\
\end{cases}$  

In [3]:
Z_variable_names = [str(c) for c in range(1, n_cells+1)]
Z = LpVariable.matrix('Z', Z_variable_names, cat='Binary')
cell_operation = np.array(Z).reshape(n_cells)
print("Cell operation variable array: ")
print(cell_operation)

Cell operation variable array: 
[Z_1 Z_2 Z_3 Z_4 Z_5 Z_6 Z_7 Z_8 Z_9 Z_10]


#### Allocation variables
The allocation variables indicate which product is allocated to which cell.  
$X_{p,\ c} \ =\ \begin{cases}
1 & \mathrm{if,} \ product\ p\ is\ allocated\ to\ be\ produced\ in\ cell\ c\\
0 & \mathrm{if,} \ product\ p\ is\ not\ allocated\ to\ be\ produced\ in\ cell\ c\\
\end{cases}$  

In [4]:
X_variable_names = [str(p)+'_'+str(c) for p in range(1, n_products+1) for c in range(1, n_cells+1)]
X = LpVariable.matrix('X', X_variable_names, cat='Binary')
allocation = np.array(X).reshape(n_products,n_cells)
print("Allocation variable Matrix: ")
print(allocation)

Allocation variable Matrix: 
[[X_1_1 X_1_2 X_1_3 X_1_4 X_1_5 X_1_6 X_1_7 X_1_8 X_1_9 X_1_10]
 [X_2_1 X_2_2 X_2_3 X_2_4 X_2_5 X_2_6 X_2_7 X_2_8 X_2_9 X_2_10]
 [X_3_1 X_3_2 X_3_3 X_3_4 X_3_5 X_3_6 X_3_7 X_3_8 X_3_9 X_3_10]
 [X_4_1 X_4_2 X_4_3 X_4_4 X_4_5 X_4_6 X_4_7 X_4_8 X_4_9 X_4_10]
 [X_5_1 X_5_2 X_5_3 X_5_4 X_5_5 X_5_6 X_5_7 X_5_8 X_5_9 X_5_10]]


### Objective
The objective is to minimize the number of cells that are operating (opened) in the next hour which can be written as:  
    __*Minimize*__ $\sum ^{n_{c}}_{1} Z_{c}$ 

In [5]:
# --- problem definition
lp_prob = LpProblem('Production Allocation Problem', LpMinimize)
# --- Objective
lp_prob += lpSum(cell_operation)

C:\ProgramData\Anaconda3\lib\site-packages\pulp\pulp.py:1198: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


### Constraints
#### Production Requirement constraints
The production of product p across all cells should be atleast the required amount.   
ie; Total amount of product p produced across all cells * production rate of product p >= requirement for product p  
$\sum ^{n_{c}}_{c\ =\ 1} X_{p,\ c} \ .\ T_{p} \ \leqslant \ D_{p} \ for\ each\ p\ \in \{1,\ n_{p}\}$

In [6]:
print("Production Requirement constraints:")
for p in range(n_products):
    print("Product "+str(p+1)+": ")
    print(lpSum(allocation[p][c] for c in range(n_cells))*T[p] >= D[p])
    lp_prob += lpSum(allocation[p][c] for c in range(n_cells))*T[p] >= D[p], \
                "Production Requirement Constraint for product" + str(p+1)

Production Requirement constraints:
Product 1: 
9*X_1_1 + 9*X_1_10 + 9*X_1_2 + 9*X_1_3 + 9*X_1_4 + 9*X_1_5 + 9*X_1_6 + 9*X_1_7 + 9*X_1_8 + 9*X_1_9 >= 11
Product 2: 
11*X_2_1 + 11*X_2_10 + 11*X_2_2 + 11*X_2_3 + 11*X_2_4 + 11*X_2_5 + 11*X_2_6 + 11*X_2_7 + 11*X_2_8 + 11*X_2_9 >= 17
Product 3: 
10*X_3_1 + 10*X_3_10 + 10*X_3_2 + 10*X_3_3 + 10*X_3_4 + 10*X_3_5 + 10*X_3_6 + 10*X_3_7 + 10*X_3_8 + 10*X_3_9 >= 16
Product 4: 
8*X_4_1 + 8*X_4_10 + 8*X_4_2 + 8*X_4_3 + 8*X_4_4 + 8*X_4_5 + 8*X_4_6 + 8*X_4_7 + 8*X_4_8 + 8*X_4_9 >= 15
Product 5: 
6*X_5_1 + 6*X_5_10 + 6*X_5_2 + 6*X_5_3 + 6*X_5_4 + 6*X_5_5 + 6*X_5_6 + 6*X_5_7 + 6*X_5_8 + 6*X_5_9 >= 12


#### Producing capability constraints
These constraints indicate whether or not cell c can produce product p. We do this by restricting value of $X_{p,\ c}$ to less than or equal to $W_{p, \ c}$. ie;  
$X_{p,\ c} \ \ \leqslant \ W_{p,\ c} \ \ for\ each\ p\ \in \{1,\ n_{p}\} \ ,\ c\ \in \{1,\ n_{c}\}$

In [7]:
print("Producing capability constraints [X_p_c]:")
for c in range(n_cells):
    for p in range(n_products):
        print(allocation[p][c] <= W[p][c])
        lp_prob += allocation[p][c] <= W[p][c], "Producing capability of cell" + str(c) + " for product" + str(p)

Producing capability constraints [X_p_c]:
X_1_1 <= 1
X_2_1 <= 1
X_3_1 <= 1
X_4_1 <= 0
X_5_1 <= 0
X_1_2 <= 1
X_2_2 <= 0
X_3_2 <= 1
X_4_2 <= 1
X_5_2 <= 1
X_1_3 <= 0
X_2_3 <= 1
X_3_3 <= 0
X_4_3 <= 0
X_5_3 <= 0
X_1_4 <= 0
X_2_4 <= 0
X_3_4 <= 1
X_4_4 <= 0
X_5_4 <= 0
X_1_5 <= 1
X_2_5 <= 1
X_3_5 <= 0
X_4_5 <= 1
X_5_5 <= 1
X_1_6 <= 1
X_2_6 <= 0
X_3_6 <= 0
X_4_6 <= 1
X_5_6 <= 1
X_1_7 <= 0
X_2_7 <= 1
X_3_7 <= 1
X_4_7 <= 1
X_5_7 <= 0
X_1_8 <= 1
X_2_8 <= 0
X_3_8 <= 1
X_4_8 <= 0
X_5_8 <= 1
X_1_9 <= 0
X_2_9 <= 1
X_3_9 <= 1
X_4_9 <= 1
X_5_9 <= 1
X_1_10 <= 1
X_2_10 <= 1
X_3_10 <= 0
X_4_10 <= 0
X_5_10 <= 0


#### Cell operation constraint
We require cell c to be open if atleast one product is being produced in it. We can define this as:
Z_{c} \ =\ max( X_{p,\ c} \ across\ all\ p) \.  Since puLP does not handle any other constraints apart from inequalities. We define maximum as a set of inequalities for each cell operation variable as follows:   
$
\begin{array}{ c }
Z_{c} \ \geqslant \ X_{1,\ c}\\
\vdots \\
Z_{c} \ \geqslant \ X_{n_{p} ,\ c}
\end{array} \\
 for\ each\ p\ \in \{1,\ n_{p}\}$

In [8]:
for c in range(n_cells):
    print("Cell operation for cell "+str(c+1)+" :")
    for p in range(n_products):
        print(cell_operation[c] >= allocation[p][c])
        lp_prob += cell_operation[c] >= allocation[p][c], \
        "Cell operation constraint for cell" + str(c) + " based on product" +str(p)

Cell operation for cell 1 :
-X_1_1 + Z_1 >= 0
-X_2_1 + Z_1 >= 0
-X_3_1 + Z_1 >= 0
-X_4_1 + Z_1 >= 0
-X_5_1 + Z_1 >= 0
Cell operation for cell 2 :
-X_1_2 + Z_2 >= 0
-X_2_2 + Z_2 >= 0
-X_3_2 + Z_2 >= 0
-X_4_2 + Z_2 >= 0
-X_5_2 + Z_2 >= 0
Cell operation for cell 3 :
-X_1_3 + Z_3 >= 0
-X_2_3 + Z_3 >= 0
-X_3_3 + Z_3 >= 0
-X_4_3 + Z_3 >= 0
-X_5_3 + Z_3 >= 0
Cell operation for cell 4 :
-X_1_4 + Z_4 >= 0
-X_2_4 + Z_4 >= 0
-X_3_4 + Z_4 >= 0
-X_4_4 + Z_4 >= 0
-X_5_4 + Z_4 >= 0
Cell operation for cell 5 :
-X_1_5 + Z_5 >= 0
-X_2_5 + Z_5 >= 0
-X_3_5 + Z_5 >= 0
-X_4_5 + Z_5 >= 0
-X_5_5 + Z_5 >= 0
Cell operation for cell 6 :
-X_1_6 + Z_6 >= 0
-X_2_6 + Z_6 >= 0
-X_3_6 + Z_6 >= 0
-X_4_6 + Z_6 >= 0
-X_5_6 + Z_6 >= 0
Cell operation for cell 7 :
-X_1_7 + Z_7 >= 0
-X_2_7 + Z_7 >= 0
-X_3_7 + Z_7 >= 0
-X_4_7 + Z_7 >= 0
-X_5_7 + Z_7 >= 0
Cell operation for cell 8 :
-X_1_8 + Z_8 >= 0
-X_2_8 + Z_8 >= 0
-X_3_8 + Z_8 >= 0
-X_4_8 + Z_8 >= 0
-X_5_8 + Z_8 >= 0
Cell operation for cell 9 :
-X_1_9 + Z_9 >= 0
-X_2_9 + Z

## Solution

In [9]:
# --- Solving
status = lp_prob.solve()
print("Status:", LpStatus[lp_prob.status])
# --- Solution
if status == 1:
    print("Minimum number of cells to be opened: " + str(lp_prob.objective.value()))
    print('\n')
    
    #--- cell wise
    open_cells = [int(v.name.split('_')[1]) for v in lp_prob.variables() if 'Z' in v.name and v.value() == 1]
    print("Cells to be opened: ")
    print(open_cells)
    for c in range(1, n_cells+1):
        cell_vars = [v.value() for v in lp_prob.variables() if 'X' in v.name and v.name.split('_')[2] == str(c)]
        if(sum(cell_vars) != 0):
            print("Cell "+str(c)+" produces products:")
            print([index+1 for index,value in enumerate(cell_vars) if value == 1])
    print('\n')

    # --- product wise
    for p in range(1, n_products+1):
        product_vars = [v.value() for v in lp_prob.variables() if 'X' in v.name and v.name.split('_')[1] == str(p)]
        print("Product "+str(p)+" is produced in Cells:")
        print([index for index,value in enumerate(product_vars) if value == 1])   

Status: Optimal
Minimum number of cells to be opened: 3


Cells to be opened: 
[2, 5, 9]
Cell 2 produces products:
[1, 3, 4, 5]
Cell 5 produces products:
[1, 2, 5]
Cell 9 produces products:
[2, 3, 4, 5]


Product 1 is produced in Cells:
[2, 5]
Product 2 is produced in Cells:
[5, 9]
Product 3 is produced in Cells:
[2, 9]
Product 4 is produced in Cells:
[2, 9]
Product 5 is produced in Cells:
[2, 5, 9]


## Conclusion Notes  
Since all three cells opened has the ability to produce product 5, solver has provided this solution. Technically, product 5 needs to be produced only in any two of these cells to meet the given constraints, hence **there are many optimal solutions** here. If there were additional constraints such as maximum amount products that can be excess or cost limit of production given production cost per each cell, we would we able to obtain a more specific solution.

## ALTERNATIVE METHOD

Defining capacity of each cell as $U_{c}$ based on information given in the problem. We can alternatively define decision variables as follows:
#### Cell operation variables
The cell operation variable is an indicator variable that denotes whether cell is open/close. (Same as earlier)
$Z_{c} \ =\ \begin{cases}
1 & \mathrm{if,} \ cell\ c\ is\ open\\
0 & \mathrm{if,} \ cell\ c\ is\ closed\\
\end{cases}$   
$for\ each\ c\ \in \{1,\ n_{c}\}$ 
#### Cell capacity usage variables 
The cell capacity usage variables denote proportion of capacity of cell $c$ used by product $p$.  
$Y_{p,\ c} \in [0, 1]\ for\ each\ p\ \in \{1,\ n_{p}\} \ ,\ c\ \in \{1,\ n_{c}\}$  
Note: $Y_{p, \ c}$ can only take values either $\frac{1}{U_{c}}$  (if product $p$ is allocated to cell $c$) or $0$

In [10]:
Z_variable_names = [str(c) for c in range(1, n_cells+1)]
Z = LpVariable.matrix('Z', Z_variable_names, cat='Binary')
cell_operation = np.array(Z).reshape(n_cells)
print("Cell operation variable array: ")
print(cell_operation)
Y_variable_names = [str(p)+'_'+str(c) for p in range(1, n_products+1) for c in range(1, n_cells+1)]
Y = LpVariable.matrix('Y', Y_variable_names, lowBound=0, upBound=1, cat='Continuous')
capacity_usage = np.array(Y).reshape(n_products,n_cells)
print("Capacity usage variable Matrix: ")
print(capacity_usage)

Cell operation variable array: 
[Z_1 Z_2 Z_3 Z_4 Z_5 Z_6 Z_7 Z_8 Z_9 Z_10]
Capacity usage variable Matrix: 
[[Y_1_1 Y_1_2 Y_1_3 Y_1_4 Y_1_5 Y_1_6 Y_1_7 Y_1_8 Y_1_9 Y_1_10]
 [Y_2_1 Y_2_2 Y_2_3 Y_2_4 Y_2_5 Y_2_6 Y_2_7 Y_2_8 Y_2_9 Y_2_10]
 [Y_3_1 Y_3_2 Y_3_3 Y_3_4 Y_3_5 Y_3_6 Y_3_7 Y_3_8 Y_3_9 Y_3_10]
 [Y_4_1 Y_4_2 Y_4_3 Y_4_4 Y_4_5 Y_4_6 Y_4_7 Y_4_8 Y_4_9 Y_4_10]
 [Y_5_1 Y_5_2 Y_5_3 Y_5_4 Y_5_5 Y_5_6 Y_5_7 Y_5_8 Y_5_9 Y_5_10]]


The parameters now include additionally the above mentioned $U_{c}$.

In [11]:
U = W.sum(axis=0)
print(U)

[3 4 1 1 4 3 3 3 4 2]


The objective remains the same.

In [12]:
# Re-declaring problem
# --- problem definition
lp_prob_alt = LpProblem('Production Allocation Problem', LpMinimize)
# --- Objective
lp_prob_alt += lpSum(cell_operation)

### Constraints
#### Production Requirement constraints
The production of product p across all cells should be atleast the required amount.   
ie; Total amount of product p produced across all cells * production rate of product p >= requirement for product p  
$\sum ^{n_{c}}_{c\ =\ 1} X_{p,\ c} \ .\ T_{p} \ \geqslant \ D_{p} \ for\ each\ p\ \in \{1,\ n_{p}\}$

In [13]:
print("Production Requirement constraints:")
for p in range(n_products):
    print("Product "+str(p+1)+": ")
    print(lpSum(capacity_usage[p][c] for c in range(n_cells))*T[p] >= D[p])
    lp_prob_alt += lpSum(capacity_usage[p][c] for c in range(n_cells))*T[p] >= D[p], \
                "Production Requirement Constraint for product" + str(p+1)

Production Requirement constraints:
Product 1: 
9*Y_1_1 + 9*Y_1_10 + 9*Y_1_2 + 9*Y_1_3 + 9*Y_1_4 + 9*Y_1_5 + 9*Y_1_6 + 9*Y_1_7 + 9*Y_1_8 + 9*Y_1_9 >= 11
Product 2: 
11*Y_2_1 + 11*Y_2_10 + 11*Y_2_2 + 11*Y_2_3 + 11*Y_2_4 + 11*Y_2_5 + 11*Y_2_6 + 11*Y_2_7 + 11*Y_2_8 + 11*Y_2_9 >= 17
Product 3: 
10*Y_3_1 + 10*Y_3_10 + 10*Y_3_2 + 10*Y_3_3 + 10*Y_3_4 + 10*Y_3_5 + 10*Y_3_6 + 10*Y_3_7 + 10*Y_3_8 + 10*Y_3_9 >= 16
Product 4: 
8*Y_4_1 + 8*Y_4_10 + 8*Y_4_2 + 8*Y_4_3 + 8*Y_4_4 + 8*Y_4_5 + 8*Y_4_6 + 8*Y_4_7 + 8*Y_4_8 + 8*Y_4_9 >= 15
Product 5: 
6*Y_5_1 + 6*Y_5_10 + 6*Y_5_2 + 6*Y_5_3 + 6*Y_5_4 + 6*Y_5_5 + 6*Y_5_6 + 6*Y_5_7 + 6*Y_5_8 + 6*Y_5_9 >= 12


#### Cell capacity limit constraints
Production in each cell c cannot exceed it's limit $U_{c}$. That is total amount of capacity used by all products made in cell c <= cell capacity.  
$U_{c}.\sum ^{n_{p}}_{p\ =\ 1} X_{p,\ c} \ \leqslant 1  \ for\ each\ c\ \in \{1,\ n_{c}\}$

In [14]:
"""
for c in range(n_cells):
    print("Cell capacity limit for cell "+str(c+1)+" :")
    print(lpSum(capacity_usage[p][c] for p in range(n_products))*U[c] <= 1)
    lp_prob_alt += lpSum(capacity_usage[p][c] for p in range(n_products))*U[c] <= 1, \
    "Cell capacity limit constraint for cell" + str(c) + " based on product" +str(p)
"""

'\nfor c in range(n_cells):\n    print("Cell capacity limit for cell "+str(c+1)+" :")\n    print(lpSum(capacity_usage[p][c] for p in range(n_products))*U[c] <= 1)\n    lp_prob_alt += lpSum(capacity_usage[p][c] for p in range(n_products))*U[c] <= 1,     "Cell capacity limit constraint for cell" + str(c) + " based on product" +str(p)\n'

#### Producing capability constraints
These constraints indicate whether or not cell c can produce product p. We do this by restricting value of $X_{p,\ c}$ to 0 if $W_{p, \ c} = 0$.ie;  
$X_{p,\ c} \ \ = 0$ if $\ W_{p,\ c} = 0$     
$for\ each\ p\ \in \{1,\ n_{p}\} \ ,\ c\ \in \{1,\ n_{c}\}$

In [15]:
print("Producing capability constraints [X_p_c]:")
for c in range(n_cells):
    for p in range(n_products):
        if W[p][c] == 0:
            print(capacity_usage[p][c] == 0)
            lp_prob_alt += capacity_usage[p][c] == 0, "Cell" + str(c) + " cannot produce product" + str(p)
        else:
            print(capacity_usage[p][c] <= 1/U[c])
            lp_prob_alt += capacity_usage[p][c] <= 1/U[c], "Cell" + str(c) + "can allocate 1/" + str(U[c]) + "capacity to product " + str(p)

Producing capability constraints [X_p_c]:
Y_1_1 <= 0.3333333333333333
Y_2_1 <= 0.3333333333333333
Y_3_1 <= 0.3333333333333333
Y_4_1 = 0
Y_5_1 = 0
Y_1_2 <= 0.25
Y_2_2 = 0
Y_3_2 <= 0.25
Y_4_2 <= 0.25
Y_5_2 <= 0.25
Y_1_3 = 0
Y_2_3 <= 1.0
Y_3_3 = 0
Y_4_3 = 0
Y_5_3 = 0
Y_1_4 = 0
Y_2_4 = 0
Y_3_4 <= 1.0
Y_4_4 = 0
Y_5_4 = 0
Y_1_5 <= 0.25
Y_2_5 <= 0.25
Y_3_5 = 0
Y_4_5 <= 0.25
Y_5_5 <= 0.25
Y_1_6 <= 0.3333333333333333
Y_2_6 = 0
Y_3_6 = 0
Y_4_6 <= 0.3333333333333333
Y_5_6 <= 0.3333333333333333
Y_1_7 = 0
Y_2_7 <= 0.3333333333333333
Y_3_7 <= 0.3333333333333333
Y_4_7 <= 0.3333333333333333
Y_5_7 = 0
Y_1_8 <= 0.3333333333333333
Y_2_8 = 0
Y_3_8 <= 0.3333333333333333
Y_4_8 = 0
Y_5_8 <= 0.3333333333333333
Y_1_9 = 0
Y_2_9 <= 0.25
Y_3_9 <= 0.25
Y_4_9 <= 0.25
Y_5_9 <= 0.25
Y_1_10 <= 0.5
Y_2_10 <= 0.5
Y_3_10 = 0
Y_4_10 = 0
Y_5_10 = 0


## Solution

In [16]:
# --- Solving
status = lp_prob_alt.solve()

In [17]:
print("Status:", LpStatus[lp_prob_alt.status])

Status: Infeasible


In [19]:
# --- Solution
if status == 1:
    print('Optimal solution obtained')
    print("Minimum number of cells to be opened: " + str(lp_prob.objective.value()))
    print('\n')
    
    #--- cell wise
    open_cells = [int(v.name.split('_')[1]) for v in lp_prob.variables() if 'Z' in v.name and v.value() == 1]
    print("Cells to be opened: ")
    print(open_cells)
    for c in range(1, n_cells+1):
        cell_vars = [v.value() for v in lp_prob.variables() if 'X' in v.name and v.name.split('_')[2] == str(c)]
        if(sum(cell_vars) != 0):
            print("Cell "+str(c)+" produces products:")
            print([index+1 for index,value in enumerate(cell_vars) if value == 1])
    print('\n')

    # --- product wise
    for p in range(1, n_products+1):
        product_vars = [v.value() for v in lp_prob.variables() if 'X' in v.name and v.name.split('_')[1] == str(p)]
        print("Product "+str(p)+" is produced in Cells:")
        print([index for index,value in enumerate(product_vars) if value == 1])   